In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir2='/Volumes/Extreme Pro 1/'
dir1='/Volumes/Extreme Pro/'

In [ ]:
yrs=np.arange(1980,2020,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    if 1==1:
        ds=xr.open_dataset(dir2+'era_5_ivtinterp_'+str(year)+'.nc')
        ivt=ds['ivt'][:,:,:]
        ds=xr.open_dataset(dir2+'era_5_iwvinterp_'+str(year)+'.nc')
        iwv=ds['iwv'][:,:,:]
        ds=xr.open_dataset(dir2+'era_5_u850interp_'+str(year)+'.nc')
        u=ds['u']
        ds=xr.open_dataset(dir2+'era_5_v850interp_'+str(year)+'.nc')
        v=ds['v']
        s=np.sqrt(u**2+v**2)

        ds=xr.open_dataset(dir2+'era_5_mfcinterp_'+str(year)+'.nc')#era5 is divergence
        mfc=ds['mfc'][:,:,:]*-1.
    if 1==0:
        ds=xr.open_dataset(dir2+'merra2.ivt.3hr.model.'+str(year)+'.nc')
        ivt=ds['ivt'][0::2,:,:]
        ds=xr.open_dataset(dir2+'merra2.iwv.3hr.model.'+str(year)+'.nc')
        iwv=ds['iwv'][0::2,:,:]
        ds=xr.open_dataset(dir3+'merra2.u850.6hr.'+str(year)+'.nc')
        u=ds['u']
        ds=xr.open_dataset(dir3+'merra2.v850.6hr.'+str(year)+'.nc')
        v=ds['v']
        s=np.sqrt(u**2+v**2)
        
        ds=xr.open_dataset(dir2+'merra2.mfc.3hr.model.'+str(year)+'.nc')#merra-2 is tendency
        mfc=ds['mfc'][0::2,:,:]
        
    #ds=xr.open_dataset(dir2+'jra55.ar.gwv2.'+str(year)+'.nc')
    #ar=ds['ar']
    
    if 1==0:
        ds=xr.open_dataset(dir2+'jra55_ivt_'+str(year)+'.nc')
        ivt=ds['ivt'].interp(lat=ar.lat.values,lon=ar.lon.values)
        ds=xr.open_dataset(dir2+'jra55_iwv_'+str(year)+'.nc')
        iwv=ds['iwv'].interp(lat=ar.lat.values,lon=ar.lon.values)
        ds=xr.open_dataset(dir2+'jra55_u850_'+str(year)+'.nc')
        u=ds['u'].interp(lat=ar.lat.values,lon=ar.lon.values)
        ds=xr.open_dataset(dir2+'jra55_v850_'+str(year)+'.nc')
        v=ds['v'].interp(lat=ar.lat.values,lon=ar.lon.values)
        s=np.sqrt(u**2+v**2)
        
    ds=xr.open_dataset(dir2+'era5.ar.tempestlr'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'era5.ar.reid500.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir1+'era5.ar.mundhenkv3.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.lorav2.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.cndl.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.acv2.'+str(year)+'.nc')
    ar=ds['ar']
    
    ivt['lon']=ar.lon.values
    iwv['lon']=ar.lon.values
    s['lon']=ar.lon.values
    mfc['lon']=ar.lon.values
        
    ivt['lat']=ar.lat.values
    iwv['lat']=ar.lat.values
    s['lat']=ar.lat.values
    mfc['lat']=ar.lat.values
    
    ivt['time']=ar.time.values
    iwv['time']=ar.time.values
    s['time']=ar.time.values
    mfc['time']=ar.time.values
    
    #weights=np.cos(np.deg2rad(ar.lat)
        
    
    ivt=ivt.where(ar==1)
    ivt_mean=ivt.mean(dim='time',skipna=True)
    iwv=iwv.where(ar==1)
    iwv_mean=iwv.mean(dim='time',skipna=True)
    s=s.where(ar==1)
    s_mean=s.mean(dim='time',skipna=True)
    mfc=mfc.where(ar==1)
    mfc_mean=mfc.mean(dim='time',skipna=True)
    
    if 1==0:
        ivt_sel1=ivt[0,:,:]
        fig=plt.figure(figsize=(18,12))
        ax=plt.sub72ot(1,1,1,projection=ccrs.PlateCarree())
        ax.coastlines(resolution='10m')
        ax.pcolormesh(ivt_sel1.lon.values,ivt_sel1.lat.values,ivt_sel1,vmin=0,vmax=800,cmap=plt.cm.seismic)
        plt.show()

        iwv_sel1=iwv[0,:,:]
        fig=plt.figure(figsize=(18,12))
        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
        ax.coastlines(resolution='10m')
        ax.pcolormesh(ivt_sel1.lon.values,ivt_sel1.lat.values,iwv_sel1,vmin=0,vmax=60,cmap=plt.cm.seismic)
        plt.show()
           
        s_sel1=s[0,:,:]
        fig=plt.figure(figsize=(18,12))
        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
        ax.coastlines(resolution='10m')
        ax.pcolormesh(ivt_sel1.lon.values,ivt_sel1.lat.values,s_sel1,vmin=0,vmax=30,cmap=plt.cm.seismic)
        plt.show()
        sys.exit()
        
    if i==0:
        ivt_concat=ivt_mean
        iwv_concat=iwv_mean
        s_concat=s_mean
        mfc_concat=mfc_mean
    else:
        ivt_concat=xr.concat([ivt_concat,ivt_mean],dim='time')
        iwv_concat=xr.concat([iwv_concat,iwv_mean],dim='time')
        s_concat=xr.concat([s_concat,s_mean],dim='time')
        mfc_concat=xr.concat([mfc_concat,mfc_mean],dim='time')

In [ ]:
dk=xr.Dataset()
dk['ivt_concat']=(('time','lat','lon'),ivt_concat.values)
dk['iwv_concat']=(('time','lat','lon'),iwv_concat.values)
dk['s_concat']=(('time','lat','lon'),s_concat.values)
dk['mfc_concat']=(('time','lat','lon'),mfc_concat.values)
dk.coords['time']=yrs
dk.coords['lat']=ivt_concat.lat.values
dk.coords['lon']=ivt_concat.lon.values

dk['lat'].attrs["units"]='degrees_north'
dk['lon'].attrs["units"]='degrees_east'

try:
    os.remove(dir_data+'synoptic_geochars_era5mk.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'synoptic_geochars_era5mk.nc',mode='w',format='NETCDF4')

In [ ]:
ds.close()
ds=xr.open_dataset(dir_data+'synoptic_geochars_era5tp.nc')
ivt_concat_era5tp=ds['ivt_concat']
iwv_concat_era5tp=ds['iwv_concat']
s_concat_era5tp=ds['s_concat']
mfc_concat_era5tp=ds['mfc_concat']

ds=xr.open_dataset(dir_data+'synoptic_geochars_era5reid.nc')
ivt_concat_era5reid=ds['ivt_concat']
iwv_concat_era5reid=ds['iwv_concat']
s_concat_era5reid=ds['s_concat']
mfc_concat_era5reid=ds['mfc_concat']

ds=xr.open_dataset(dir_data+'synoptic_geochars_era5mk.nc')
ivt_concat_era5mk=ds['ivt_concat']
iwv_concat_era5mk=ds['iwv_concat']
s_concat_era5mk=ds['s_concat']
mfc_concat_era5mk=ds['mfc_concat']

ds=xr.open_dataset(dir_data+'synoptic_geochars_merra2lo.nc')
ivt_concat_merra2lo=ds['ivt_concat']
iwv_concat_merra2lo=ds['iwv_concat']
s_concat_merra2lo=ds['s_concat']
mfc_concat_merra2lo=ds['mfc_concat']

ds=xr.open_dataset(dir_data+'synoptic_geochars_merra2cndl.nc')
ivt_concat_merra2cndl=ds['ivt_concat']
iwv_concat_merra2cndl=ds['iwv_concat']
s_concat_merra2cndl=ds['s_concat']
mfc_concat_merra2cndl=ds['mfc_concat']

ds=xr.open_dataset(dir_data+'synoptic_geochars_merra2ac.nc')
ivt_concat_merra2ac=ds['ivt_concat']
iwv_concat_merra2ac=ds['iwv_concat']
s_concat_merra2ac=ds['s_concat']
mfc_concat_merra2ac=ds['mfc_concat']

ds=xr.open_dataset(dir_data+'synoptic_geochars_jra55gw.nc')
ivt_concat_jra55gw=ds['ivt_concat']
iwv_concat_jra55gw=ds['iwv_concat']
s_concat_jra55gw=ds['s_concat']

In [ ]:
lons_west=[x for x in ivt_concat_jra55gw.lon.values if x<-90]
lons_east=[x for x in ivt_concat_jra55gw.lon.values if x>=90]
sel_west=ivt_concat_jra55gw.sel(lon=lons_west)
sel_east=ivt_concat_jra55gw.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,ivt_concat_jra55gw,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5tp.lat.values,lon=ivt_concat_era5tp.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_ivt=interp.sel(lon=lons_center)


sel_west=iwv_concat_jra55gw.sel(lon=lons_west)
sel_east=iwv_concat_jra55gw.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,iwv_concat_jra55gw,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5tp.lat.values,lon=ivt_concat_era5tp.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_iwv=interp.sel(lon=lons_center)


sel_west=s_concat_jra55gw.sel(lon=lons_west)
sel_east=s_concat_jra55gw.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,s_concat_jra55gw,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5tp.lat.values,lon=ivt_concat_era5tp.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_s=interp.sel(lon=lons_center)

In [ ]:
yrs=np.arange(1980,2020,1)
slopes_array_ivt_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_ivt_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_iwv_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_iwv_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_s_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_s_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_mfc_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_mfc_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])

slopes_array_ivt_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_ivt_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_iwv_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_iwv_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_s_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_s_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_mfc_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_mfc_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])

slopes_array_ivt_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_ivt_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_iwv_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_iwv_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_s_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_s_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_mfc_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_mfc_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])


slopes_array_ivt_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_ivt_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_iwv_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_iwv_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_s_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_s_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_mfc_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_mfc_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])

slopes_array_ivt_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_ivt_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_iwv_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_iwv_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_s_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_s_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_mfc_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_mfc_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])

slopes_array_ivt_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_ivt_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_iwv_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_iwv_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_s_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_s_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_mfc_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_mfc_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])


slopes_array_ivt_jra55gw=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_ivt_jra55gw=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_iwv_jra55gw=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_iwv_jra55gw=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_s_jra55gw=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_s_jra55gw=xr.zeros_like(ivt_concat_era5tp[0,:,:])
size=20
for j in range(ivt_concat_era5tp.lat.size):
    print(j)
    for k in range(ivt_concat_era5tp.lon.size):
        time_series=ivt_concat_era5tp[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_era5tp[j,k]=r[0]
            sigs_array_ivt_era5tp[j,k]=r[3]
        else:
            slopes_array_ivt_era5tp[j,k]=np.nan
            sigs_array_ivt_era5tp[j,k]=np.nan
            
        time_series=iwv_concat_era5tp[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_era5tp[j,k]=r[0]
            sigs_array_iwv_era5tp[j,k]=r[3]
        else:
            slopes_array_iwv_era5tp[j,k]=np.nan
            sigs_array_iwv_era5tp[j,k]=np.nan
            
        time_series=s_concat_era5tp[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_era5tp[j,k]=r[0]
            sigs_array_s_era5tp[j,k]=r[3]
        else:
            slopes_array_s_era5tp[j,k]=np.nan
            sigs_array_s_era5tp[j,k]=np.nan
            
        time_series=mfc_concat_era5tp[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_era5tp[j,k]=r[0]
            sigs_array_mfc_era5tp[j,k]=r[3]
        else:
            slopes_array_mfc_era5tp[j,k]=np.nan
            sigs_array_mfc_era5tp[j,k]=np.nan
            
        ######
        
        time_series=ivt_concat_era5reid[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_era5reid[j,k]=r[0]
            sigs_array_ivt_era5reid[j,k]=r[3]
        else:
            slopes_array_ivt_era5reid[j,k]=np.nan
            sigs_array_ivt_era5reid[j,k]=np.nan
            
        time_series=iwv_concat_era5reid[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_era5reid[j,k]=r[0]
            sigs_array_iwv_era5reid[j,k]=r[3]
        else:
            slopes_array_iwv_era5reid[j,k]=np.nan
            sigs_array_iwv_era5reid[j,k]=np.nan
            
        time_series=s_concat_era5reid[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_era5reid[j,k]=r[0]
            sigs_array_s_era5reid[j,k]=r[3]
        else:
            slopes_array_s_era5reid[j,k]=np.nan
            sigs_array_s_era5reid[j,k]=np.nan
            
        time_series=mfc_concat_era5reid[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_era5reid[j,k]=r[0]
            sigs_array_mfc_era5reid[j,k]=r[3]
        else:
            slopes_array_mfc_era5reid[j,k]=np.nan
            sigs_array_mfc_era5reid[j,k]=np.nan
            
        #####
        
        time_series=ivt_concat_era5mk[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_era5mk[j,k]=r[0]
            sigs_array_ivt_era5mk[j,k]=r[3]
        else:
            slopes_array_ivt_era5mk[j,k]=np.nan
            sigs_array_ivt_era5mk[j,k]=np.nan
            
        time_series=iwv_concat_era5mk[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_era5mk[j,k]=r[0]
            sigs_array_iwv_era5mk[j,k]=r[3]
        else:
            slopes_array_iwv_era5mk[j,k]=np.nan
            sigs_array_iwv_era5mk[j,k]=np.nan
            
        time_series=s_concat_era5mk[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_era5mk[j,k]=r[0]
            sigs_array_s_era5mk[j,k]=r[3]
        else:
            slopes_array_s_era5mk[j,k]=np.nan
            sigs_array_s_era5mk[j,k]=np.nan
            
        time_series=mfc_concat_era5mk[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_era5mk[j,k]=r[0]
            sigs_array_mfc_era5mk[j,k]=r[3]
        else:
            slopes_array_mfc_era5mk[j,k]=np.nan
            sigs_array_mfc_era5mk[j,k]=np.nan
            
        ##########
        
        time_series=ivt_concat_merra2lo[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_merra2lo[j,k]=r[0]
            sigs_array_ivt_merra2lo[j,k]=r[3]
        else:
            slopes_array_ivt_merra2lo[j,k]=np.nan
            sigs_array_ivt_merra2lo[j,k]=np.nan
            
        time_series=iwv_concat_merra2lo[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_merra2lo[j,k]=r[0]
            sigs_array_iwv_merra2lo[j,k]=r[3]
        else:
            slopes_array_iwv_merra2lo[j,k]=np.nan
            sigs_array_iwv_merra2lo[j,k]=np.nan
            
        time_series=s_concat_merra2lo[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_merra2lo[j,k]=r[0]
            sigs_array_s_merra2lo[j,k]=r[3]
        else:
            slopes_array_s_merra2lo[j,k]=np.nan
            sigs_array_s_merra2lo[j,k]=np.nan
            
        time_series=mfc_concat_merra2lo[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_merra2lo[j,k]=r[0]
            sigs_array_mfc_merra2lo[j,k]=r[3]
        else:
            slopes_array_mfc_merra2lo[j,k]=np.nan
            sigs_array_mfc_merra2lo[j,k]=np.nan
            
        #####
        
        time_series=ivt_concat_merra2cndl[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_merra2cndl[j,k]=r[0]
            sigs_array_ivt_merra2cndl[j,k]=r[3]
        else:
            slopes_array_ivt_merra2cndl[j,k]=np.nan
            sigs_array_ivt_merra2cndl[j,k]=np.nan
            
        time_series=iwv_concat_merra2cndl[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_merra2cndl[j,k]=r[0]
            sigs_array_iwv_merra2cndl[j,k]=r[3]
        else:
            slopes_array_iwv_merra2cndl[j,k]=np.nan
            sigs_array_iwv_merra2cndl[j,k]=np.nan
            
        time_series=s_concat_merra2cndl[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_merra2cndl[j,k]=r[0]
            sigs_array_s_merra2cndl[j,k]=r[3]
        else:
            slopes_array_s_merra2cndl[j,k]=np.nan
            sigs_array_s_merra2cndl[j,k]=np.nan
            
        time_series=mfc_concat_merra2cndl[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_merra2cndl[j,k]=r[0]
            sigs_array_mfc_merra2cndl[j,k]=r[3]
        else:
            slopes_array_mfc_merra2cndl[j,k]=np.nan
            sigs_array_mfc_merra2cndl[j,k]=np.nan
            
        #####
        
        time_series=ivt_concat_merra2ac[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_merra2ac[j,k]=r[0]
            sigs_array_ivt_merra2ac[j,k]=r[3]
        else:
            slopes_array_ivt_merra2ac[j,k]=np.nan
            sigs_array_ivt_merra2ac[j,k]=np.nan
            
        time_series=iwv_concat_merra2ac[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_merra2ac[j,k]=r[0]
            sigs_array_iwv_merra2ac[j,k]=r[3]
        else:
            slopes_array_iwv_merra2ac[j,k]=np.nan
            sigs_array_iwv_merra2ac[j,k]=np.nan
            
        time_series=s_concat_merra2ac[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_merra2ac[j,k]=r[0]
            sigs_array_s_merra2ac[j,k]=r[3]
        else:
            slopes_array_s_merra2ac[j,k]=np.nan
            sigs_array_s_merra2ac[j,k]=np.nan
            
        time_series=mfc_concat_merra2ac[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_merra2ac[j,k]=r[0]
            sigs_array_mfc_merra2ac[j,k]=r[3]
        else:
            slopes_array_mfc_merra2ac[j,k]=np.nan
            sigs_array_mfc_merra2ac[j,k]=np.nan
            
        ############
        
        time_series=fullres_ivt[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_jra55gw[j,k]=r[0]
            sigs_array_ivt_jra55gw[j,k]=r[3]
        else:
            slopes_array_ivt_jra55gw[j,k]=np.nan
            sigs_array_ivt_jra55gw[j,k]=np.nan
            
        time_series=fullres_iwv[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_jra55gw[j,k]=r[0]
            sigs_array_iwv_jra55gw[j,k]=r[3]
        else:
            slopes_array_iwv_jra55gw[j,k]=np.nan
            sigs_array_iwv_jra55gw[j,k]=np.nan
            
        time_series=fullres_s[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_jra55gw[j,k]=r[0]
            sigs_array_s_jra55gw[j,k]=r[3]
        else:
            slopes_array_s_jra55gw[j,k]=np.nan
            sigs_array_s_jra55gw[j,k]=np.nan    

In [ ]:
dk=xr.Dataset()
dk['slopes_array_ivt_era5tp']=(('lat','lon'),slopes_array_ivt_era5tp.values)
dk['slopes_array_ivt_era5reid']=(('lat','lon'),slopes_array_ivt_era5reid.values)
dk['slopes_array_ivt_era5mk']=(('lat','lon'),slopes_array_ivt_era5mk.values)
dk['slopes_array_ivt_merra2lo']=(('lat','lon'),slopes_array_ivt_merra2lo.values)
dk['slopes_array_ivt_merra2cndl']=(('lat','lon'),slopes_array_ivt_merra2cndl.values)
dk['slopes_array_ivt_merra2ac']=(('lat','lon'),slopes_array_ivt_merra2ac.values)
dk['slopes_array_ivt_jra55gw']=(('lat','lon'),slopes_array_ivt_jra55gw.values)

dk['sigs_array_ivt_era5tp']=(('lat','lon'),sigs_array_ivt_era5tp.values)
dk['sigs_array_ivt_era5reid']=(('lat','lon'),sigs_array_ivt_era5reid.values)
dk['sigs_array_ivt_era5mk']=(('lat','lon'),sigs_array_ivt_era5mk.values)
dk['sigs_array_ivt_merra2lo']=(('lat','lon'),sigs_array_ivt_merra2lo.values)
dk['sigs_array_ivt_merra2cndl']=(('lat','lon'),sigs_array_ivt_merra2cndl.values)
dk['sigs_array_ivt_merra2ac']=(('lat','lon'),sigs_array_ivt_merra2ac.values)
dk['sigs_array_ivt_jra55gw']=(('lat','lon'),sigs_array_ivt_jra55gw.values)


dk['slopes_array_iwv_era5tp']=(('lat','lon'),slopes_array_iwv_era5tp.values)
dk['slopes_array_iwv_era5reid']=(('lat','lon'),slopes_array_iwv_era5reid.values)
dk['slopes_array_iwv_era5mk']=(('lat','lon'),slopes_array_iwv_era5mk.values)
dk['slopes_array_iwv_merra2lo']=(('lat','lon'),slopes_array_iwv_merra2lo.values)
dk['slopes_array_iwv_merra2cndl']=(('lat','lon'),slopes_array_iwv_merra2cndl.values)
dk['slopes_array_iwv_merra2ac']=(('lat','lon'),slopes_array_iwv_merra2ac.values)
dk['slopes_array_iwv_jra55gw']=(('lat','lon'),slopes_array_iwv_jra55gw.values)

dk['sigs_array_iwv_era5tp']=(('lat','lon'),sigs_array_iwv_era5tp.values)
dk['sigs_array_iwv_era5reid']=(('lat','lon'),sigs_array_iwv_era5reid.values)
dk['sigs_array_iwv_era5mk']=(('lat','lon'),sigs_array_iwv_era5mk.values)
dk['sigs_array_iwv_merra2lo']=(('lat','lon'),sigs_array_iwv_merra2lo.values)
dk['sigs_array_iwv_merra2cndl']=(('lat','lon'),sigs_array_iwv_merra2cndl.values)
dk['sigs_array_iwv_merra2ac']=(('lat','lon'),sigs_array_iwv_merra2ac.values)
dk['sigs_array_iwv_jra55gw']=(('lat','lon'),sigs_array_iwv_jra55gw.values)


dk['slopes_array_s_era5tp']=(('lat','lon'),slopes_array_s_era5tp.values)
dk['slopes_array_s_era5reid']=(('lat','lon'),slopes_array_s_era5reid.values)
dk['slopes_array_s_era5mk']=(('lat','lon'),slopes_array_s_era5mk.values)
dk['slopes_array_s_merra2lo']=(('lat','lon'),slopes_array_s_merra2lo.values)
dk['slopes_array_s_merra2cndl']=(('lat','lon'),slopes_array_s_merra2cndl.values)
dk['slopes_array_s_merra2ac']=(('lat','lon'),slopes_array_s_merra2ac.values)
dk['slopes_array_s_jra55gw']=(('lat','lon'),slopes_array_s_jra55gw.values)

dk['sigs_array_s_era5tp']=(('lat','lon'),sigs_array_s_era5tp.values)
dk['sigs_array_s_era5reid']=(('lat','lon'),sigs_array_s_era5reid.values)
dk['sigs_array_s_era5mk']=(('lat','lon'),sigs_array_s_era5mk.values)
dk['sigs_array_s_merra2lo']=(('lat','lon'),sigs_array_s_merra2lo.values)
dk['sigs_array_s_merra2cndl']=(('lat','lon'),sigs_array_s_merra2cndl.values)
dk['sigs_array_s_merra2ac']=(('lat','lon'),sigs_array_s_merra2ac.values)
dk['sigs_array_s_jra55gw']=(('lat','lon'),sigs_array_s_jra55gw.values)


dk['slopes_array_mfc_era5tp']=(('lat','lon'),slopes_array_mfc_era5tp.values)
dk['slopes_array_mfc_era5reid']=(('lat','lon'),slopes_array_mfc_era5reid.values)
dk['slopes_array_mfc_era5mk']=(('lat','lon'),slopes_array_mfc_era5mk.values)
dk['slopes_array_mfc_merra2lo']=(('lat','lon'),slopes_array_mfc_merra2lo.values)
dk['slopes_array_mfc_merra2cndl']=(('lat','lon'),slopes_array_mfc_merra2cndl.values)
dk['slopes_array_mfc_merra2ac']=(('lat','lon'),slopes_array_mfc_merra2ac.values)

dk['sigs_array_mfc_era5tp']=(('lat','lon'),sigs_array_mfc_era5tp.values)
dk['sigs_array_mfc_era5reid']=(('lat','lon'),sigs_array_mfc_era5reid.values)
dk['sigs_array_mfc_era5mk']=(('lat','lon'),sigs_array_mfc_era5mk.values)
dk['sigs_array_mfc_merra2lo']=(('lat','lon'),sigs_array_mfc_merra2lo.values)
dk['sigs_array_mfc_merra2cndl']=(('lat','lon'),sigs_array_mfc_merra2cndl.values)
dk['sigs_array_mfc_merra2ac']=(('lat','lon'),sigs_array_mfc_merra2ac.values)

dk.coords['lat']=slopes_array_ivt_era5tp.lat.values
dk.coords['lon']=slopes_array_ivt_era5tp.lon.values

dk['lat'].attrs["units"]='degrees_north'
dk['lon'].attrs["units"]='degrees_east'

try:
    os.remove(dir_data+'synoptic_geotrends_arrays_artmip.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'synoptic_geotrends_arrays_artmip.nc',mode='w',format='NETCDF4')

In [ ]:
ds=xr.open_dataset(dir_data+'synoptic_geotrends_arrays_artmip.nc')

slopes_array_ivt_era5tp=ds['slopes_array_ivt_era5tp']
slopes_array_ivt_era5reid=ds['slopes_array_ivt_era5reid']
slopes_array_ivt_era5mk=ds['slopes_array_ivt_era5mk']
slopes_array_ivt_merra2lo=ds['slopes_array_ivt_merra2lo']
slopes_array_ivt_merra2cndl=ds['slopes_array_ivt_merra2cndl']
slopes_array_ivt_merra2ac=ds['slopes_array_ivt_merra2ac']
slopes_array_ivt_jra55gw=ds['slopes_array_ivt_jra55gw']

slopes_array_iwv_era5tp=ds['slopes_array_iwv_era5tp']
slopes_array_iwv_era5reid=ds['slopes_array_iwv_era5reid']
slopes_array_iwv_era5mk=ds['slopes_array_iwv_era5mk']
slopes_array_iwv_merra2lo=ds['slopes_array_iwv_merra2lo']
slopes_array_iwv_merra2cndl=ds['slopes_array_iwv_merra2cndl']
slopes_array_iwv_merra2ac=ds['slopes_array_iwv_merra2ac']
slopes_array_iwv_jra55gw=ds['slopes_array_iwv_jra55gw']

slopes_array_s_era5tp=ds['slopes_array_s_era5tp']
slopes_array_s_era5reid=ds['slopes_array_s_era5reid']
slopes_array_s_era5mk=ds['slopes_array_s_era5mk']
slopes_array_s_merra2lo=ds['slopes_array_s_merra2lo']
slopes_array_s_merra2cndl=ds['slopes_array_s_merra2cndl']
slopes_array_s_merra2ac=ds['slopes_array_s_merra2ac']
slopes_array_s_jra55gw=ds['slopes_array_s_jra55gw']

slopes_array_mfc_era5tp=ds['slopes_array_mfc_era5tp']
slopes_array_mfc_era5reid=ds['slopes_array_mfc_era5reid']
slopes_array_mfc_era5mk=ds['slopes_array_mfc_era5mk']
slopes_array_mfc_merra2lo=ds['slopes_array_mfc_merra2lo']
slopes_array_mfc_merra2cndl=ds['slopes_array_mfc_merra2cndl']
slopes_array_mfc_merra2ac=ds['slopes_array_mfc_merra2ac']


sigs_array_ivt_era5tp=ds['sigs_array_ivt_era5tp']
sigs_array_ivt_era5reid=ds['sigs_array_ivt_era5reid']
sigs_array_ivt_era5mk=ds['sigs_array_ivt_era5mk']
sigs_array_ivt_merra2lo=ds['sigs_array_ivt_merra2lo']
sigs_array_ivt_merra2cndl=ds['sigs_array_ivt_merra2cndl']
sigs_array_ivt_merra2ac=ds['sigs_array_ivt_merra2ac']
sigs_array_ivt_jra55gw=ds['sigs_array_ivt_jra55gw']

sigs_array_iwv_era5tp=ds['sigs_array_iwv_era5tp']
sigs_array_iwv_era5reid=ds['sigs_array_iwv_era5reid']
sigs_array_iwv_era5mk=ds['sigs_array_iwv_era5mk']
sigs_array_iwv_merra2lo=ds['sigs_array_iwv_merra2lo']
sigs_array_iwv_merra2cndl=ds['sigs_array_iwv_merra2cndl']
sigs_array_iwv_merra2ac=ds['sigs_array_iwv_merra2ac']
sigs_array_iwv_jra55gw=ds['sigs_array_iwv_jra55gw']

sigs_array_s_era5tp=ds['sigs_array_s_era5tp']
sigs_array_s_era5reid=ds['sigs_array_s_era5reid']
sigs_array_s_era5mk=ds['sigs_array_s_era5mk']
sigs_array_s_merra2lo=ds['sigs_array_s_merra2lo']
sigs_array_s_merra2cndl=ds['sigs_array_s_merra2cndl']
sigs_array_s_merra2ac=ds['sigs_array_s_merra2ac']
sigs_array_s_jra55gw=ds['sigs_array_s_jra55gw']

sigs_array_mfc_era5tp=ds['sigs_array_mfc_era5tp']
sigs_array_mfc_era5reid=ds['sigs_array_mfc_era5reid']
sigs_array_mfc_era5mk=ds['sigs_array_mfc_era5mk']
sigs_array_mfc_merra2lo=ds['sigs_array_mfc_merra2lo']
sigs_array_mfc_merra2cndl=ds['sigs_array_mfc_merra2cndl']
sigs_array_mfc_merra2ac=ds['sigs_array_mfc_merra2ac']

In [ ]:
a=0.05#/sum_factor

p_values=sigs_array_ivt_era5tp.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_ivt_era5tp=sigs_array_ivt_era5tp.where(sigs_array_ivt_era5tp<=p_final)

p_values=sigs_array_ivt_era5reid.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_ivt_era5reid=sigs_array_ivt_era5reid.where(sigs_array_ivt_era5reid<=p_final)

p_values=sigs_array_ivt_era5mk.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_ivt_era5mk=sigs_array_ivt_era5mk.where(sigs_array_ivt_era5mk<=p_final)

########


p_values=sigs_array_ivt_merra2lo.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_ivt_merra2lo=sigs_array_ivt_merra2lo.where(sigs_array_ivt_merra2lo<=p_final)

p_values=sigs_array_ivt_merra2cndl.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_ivt_merra2cndl=sigs_array_ivt_merra2cndl.where(sigs_array_ivt_merra2cndl<=p_final)

p_values=sigs_array_ivt_merra2ac.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_ivt_merra2ac=sigs_array_ivt_merra2ac.where(sigs_array_ivt_merra2ac<=p_final)

######

p_values=sigs_array_ivt_jra55gw.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_ivt_jra55gw=sigs_array_ivt_jra55gw.where(sigs_array_ivt_jra55gw<=p_final)

In [ ]:
num_sets1=xr.ones_like(slopes_array_ivt_era5tp).where(slopes_array_ivt_era5tp>-1000)
num_sets1=num_sets1.fillna(0)
num_sets2=xr.ones_like(slopes_array_ivt_era5reid).where(slopes_array_ivt_era5reid>-1000)
num_sets2=num_sets2.fillna(0)
num_sets3=xr.ones_like(slopes_array_ivt_era5mk).where(slopes_array_ivt_era5mk>-1000)
num_sets3=num_sets3.fillna(0)

num_sets4=xr.ones_like(slopes_array_ivt_merra2lo).where(slopes_array_ivt_merra2lo>-1000)
num_sets4=num_sets4.fillna(0)
num_sets5=xr.ones_like(slopes_array_ivt_merra2cndl).where(slopes_array_ivt_merra2cndl>-1000)
num_sets5=num_sets5.fillna(0)
num_sets6=xr.ones_like(slopes_array_ivt_merra2ac).where(slopes_array_ivt_merra2ac>-1000)
num_sets6=num_sets6.fillna(0)

num_sets7=xr.ones_like(slopes_array_ivt_jra55gw).where(slopes_array_ivt_jra55gw>-1000)
num_sets7=num_sets7.fillna(0)

num_sets=num_sets1+num_sets2+num_sets3+num_sets4+num_sets5+num_sets6+num_sets7
field=num_sets


fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(0,10,1),cmap=plt.cm.Greens,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-2,2.2,0.2),cmap=plt.cm.seismic,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('# datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False
#ax.set_ylim(-60,70)

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{\#}$'+' '+r'$\bf{datasets:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{AR}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)

plt.show()

In [ ]:
# datasets - 
# datasets - 3 AR variants
# datasets - 2 EIVT variants

test1=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_ivt_era5tp<=0.05).where(slopes_array_ivt_era5tp>0).fillna(0)
test2=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_ivt_era5reid<=0.05).where(slopes_array_ivt_era5reid>0).fillna(0)
test3=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_ivt_era5mk<=0.05).where(slopes_array_ivt_era5mk>0).fillna(0)

test4=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_ivt_merra2lo<=0.05).where(slopes_array_ivt_merra2lo>0).fillna(0)
test5=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_ivt_merra2cndl<=0.05).where(slopes_array_ivt_merra2cndl>0).fillna(0)
test6=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_ivt_merra2ac<=0.05).where(slopes_array_ivt_merra2ac>0).fillna(0)

test7=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_final_ivt_jra55gw<=0.05).where(slopes_array_ivt_jra55gw>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7)/num_sets*100.
field=test_all


test1a=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_ivt_era5tp<=0.05).where(slopes_array_ivt_era5tp<0).fillna(0)
test2a=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_ivt_era5reid<=0.05).where(slopes_array_ivt_era5reid<0).fillna(0)
test3a=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_ivt_era5mk<=0.05).where(slopes_array_ivt_era5mk<0).fillna(0)

test4a=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_ivt_merra2lo<=0.05).where(slopes_array_ivt_merra2lo<0).fillna(0)
test5a=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_ivt_merra2cndl<=0.05).where(slopes_array_ivt_merra2cndl<0).fillna(0)
test6a=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_ivt_merra2ac<=0.05).where(slopes_array_ivt_merra2ac<0).fillna(0)

test7a=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_final_ivt_jra55gw<=0.05).where(slopes_array_ivt_jra55gw<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

#######

test1b=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_ivt_era5tp<=0.05).where(slopes_array_ivt_era5tp>0).fillna(0)
test2b=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_ivt_era5reid<=0.05).where(slopes_array_ivt_era5reid>0).fillna(0)
test3b=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_ivt_era5mk<=0.05).where(slopes_array_ivt_era5mk>0).fillna(0)

test4b=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_ivt_merra2lo<=0.05).where(slopes_array_ivt_merra2lo>0).fillna(0)
test5b=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_ivt_merra2cndl<=0.05).where(slopes_array_ivt_merra2cndl>0).fillna(0)
test6b=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_ivt_merra2ac<=0.05).where(slopes_array_ivt_merra2ac>0).fillna(0)

test7b=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_ivt_jra55gw<=0.05).where(slopes_array_ivt_jra55gw>0).fillna(0)

test_allb=(test1b+test2b+test3b+test4b+test5b+test6b+test7b)/num_sets*100.


test1c=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_ivt_era5tp<=0.05).where(slopes_array_ivt_era5tp<0).fillna(0)
test2c=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_ivt_era5reid<=0.05).where(slopes_array_ivt_era5reid<0).fillna(0)
test3c=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_ivt_era5mk<=0.05).where(slopes_array_ivt_era5mk<0).fillna(0)

test4c=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_ivt_merra2lo<=0.05).where(slopes_array_ivt_merra2lo<0).fillna(0)
test5c=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_ivt_merra2cndl<=0.05).where(slopes_array_ivt_merra2cndl<0).fillna(0)
test6c=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_ivt_merra2ac<=0.05).where(slopes_array_ivt_merra2ac<0).fillna(0)

test7c=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_ivt_jra55gw<=0.05).where(slopes_array_ivt_jra55gw<0).fillna(0)

test_allc=(test1c+test2c+test3c+test4c+test5c+test6c+test7c)/num_sets*100.
fieldc=test_allc

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)
test_allb=test_allb.where(num_sets>=2)
test_allc=test_allc.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))

cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1.,levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

X=4
for j in range(int(field.lat.size/X)):
    for k in range(int(field.lon.size/X)):
        if test_allb[X*j,X*k]>=66.66 or test_allc[X*j,X*k]>=66.66:
            ax.plot(field.lon[X*k],field.lat[X*j],marker='o',color='k',markersize=2,alpha=0.5)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{Trends:}$'+' '+r'$\bf{ARTMIP}$'+' '+r'$\bf{Tier}$'+' '+r'$\bf{2}$'+' '+r'$\bf{sample}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{IVT}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_sel_1.png')

In [ ]:
# datasets - 
# datasets - 3 AR variants
# datasets - 2 EIVT variants

test1=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_ivt_era5tp<=0.05).where(slopes_array_ivt_era5tp>0).fillna(0)
test2=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_ivt_era5reid<=0.05).where(slopes_array_ivt_era5reid>0).fillna(0)
test3=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_ivt_era5mk<=0.05).where(slopes_array_ivt_era5mk>0).fillna(0)

test4=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_ivt_merra2lo<=0.05).where(slopes_array_ivt_merra2lo>0).fillna(0)
test5=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_ivt_merra2cndl<=0.05).where(slopes_array_ivt_merra2cndl>0).fillna(0)
test6=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_ivt_merra2ac<=0.05).where(slopes_array_ivt_merra2ac>0).fillna(0)

test7=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_ivt_jra55gw<=0.05).where(slopes_array_ivt_jra55gw>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7)/num_sets*100.
field=test_all


test1a=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_ivt_era5tp<=0.05).where(slopes_array_ivt_era5tp<0).fillna(0)
test2a=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_ivt_era5reid<=0.05).where(slopes_array_ivt_era5reid<0).fillna(0)
test3a=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_ivt_era5mk<=0.05).where(slopes_array_ivt_era5mk<0).fillna(0)

test4a=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_ivt_merra2lo<=0.05).where(slopes_array_ivt_merra2lo<0).fillna(0)
test5a=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_ivt_merra2cndl<=0.05).where(slopes_array_ivt_merra2cndl<0).fillna(0)
test6a=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_ivt_merra2ac<=0.05).where(slopes_array_ivt_merra2ac<0).fillna(0)

test7a=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_ivt_jra55gw<=0.05).where(slopes_array_ivt_jra55gw<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)


field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))

cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1.,levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{Trends:}$'+' '+r'$\bf{ARTMIP}$'+' '+r'$\bf{Tier}$'+' '+r'$\bf{2}$'+' '+r'$\bf{sample}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{IVT}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_sel_1a.png')

In [ ]:
a=0.05#/sum_factor

p_values=sigs_array_iwv_era5tp.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_iwv_era5tp=sigs_array_iwv_era5tp.where(sigs_array_iwv_era5tp<=p_final)

p_values=sigs_array_iwv_era5reid.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_iwv_era5reid=sigs_array_iwv_era5reid.where(sigs_array_iwv_era5reid<=p_final)

p_values=sigs_array_iwv_era5mk.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_iwv_era5mk=sigs_array_iwv_era5mk.where(sigs_array_iwv_era5mk<=p_final)

########


p_values=sigs_array_iwv_merra2lo.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_iwv_merra2lo=sigs_array_iwv_merra2lo.where(sigs_array_iwv_merra2lo<=p_final)

p_values=sigs_array_iwv_merra2cndl.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_iwv_merra2cndl=sigs_array_iwv_merra2cndl.where(sigs_array_iwv_merra2cndl<=p_final)

p_values=sigs_array_iwv_merra2ac.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_iwv_merra2ac=sigs_array_iwv_merra2ac.where(sigs_array_iwv_merra2ac<=p_final)

######

p_values=sigs_array_iwv_jra55gw.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_iwv_jra55gw=sigs_array_iwv_jra55gw.where(sigs_array_iwv_jra55gw<=p_final)

In [ ]:
# datasets - 
# datasets - 3 AR variants
# datasets - 2 EIVT variants

test1=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_iwv_era5tp<=0.05).where(slopes_array_iwv_era5tp>0).fillna(0)
test2=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_iwv_era5reid<=0.05).where(slopes_array_iwv_era5reid>0).fillna(0)
test3=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_iwv_era5mk<=0.05).where(slopes_array_iwv_era5mk>0).fillna(0)

test4=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_iwv_merra2lo<=0.05).where(slopes_array_iwv_merra2lo>0).fillna(0)
test5=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_iwv_merra2cndl<=0.05).where(slopes_array_iwv_merra2cndl>0).fillna(0)
test6=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_iwv_merra2ac<=0.05).where(slopes_array_iwv_merra2ac>0).fillna(0)

test7=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_final_iwv_jra55gw<=0.05).where(slopes_array_iwv_jra55gw>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7)/num_sets*100.
field=test_all


test1a=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_iwv_era5tp<=0.05).where(slopes_array_iwv_era5tp<0).fillna(0)
test2a=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_iwv_era5reid<=0.05).where(slopes_array_iwv_era5reid<0).fillna(0)
test3a=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_iwv_era5mk<=0.05).where(slopes_array_iwv_era5mk<0).fillna(0)

test4a=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_iwv_merra2lo<=0.05).where(slopes_array_iwv_merra2lo<0).fillna(0)
test5a=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_iwv_merra2cndl<=0.05).where(slopes_array_iwv_merra2cndl<0).fillna(0)
test6a=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_iwv_merra2ac<=0.05).where(slopes_array_iwv_merra2ac<0).fillna(0)

test7a=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_final_iwv_jra55gw<=0.05).where(slopes_array_iwv_jra55gw<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

#######

test1b=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_iwv_era5tp<=0.05).where(slopes_array_iwv_era5tp>0).fillna(0)
test2b=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_iwv_era5reid<=0.05).where(slopes_array_iwv_era5reid>0).fillna(0)
test3b=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_iwv_era5mk<=0.05).where(slopes_array_iwv_era5mk>0).fillna(0)

test4b=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_iwv_merra2lo<=0.05).where(slopes_array_iwv_merra2lo>0).fillna(0)
test5b=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_iwv_merra2cndl<=0.05).where(slopes_array_iwv_merra2cndl>0).fillna(0)
test6b=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_iwv_merra2ac<=0.05).where(slopes_array_iwv_merra2ac>0).fillna(0)

test7b=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_iwv_jra55gw<=0.05).where(slopes_array_iwv_jra55gw>0).fillna(0)

test_allb=(test1b+test2b+test3b+test4b+test5b+test6b+test7b)/num_sets*100.


test1c=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_iwv_era5tp<=0.05).where(slopes_array_iwv_era5tp<0).fillna(0)
test2c=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_iwv_era5reid<=0.05).where(slopes_array_iwv_era5reid<0).fillna(0)
test3c=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_iwv_era5mk<=0.05).where(slopes_array_iwv_era5mk<0).fillna(0)

test4c=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_iwv_merra2lo<=0.05).where(slopes_array_iwv_merra2lo<0).fillna(0)
test5c=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_iwv_merra2cndl<=0.05).where(slopes_array_iwv_merra2cndl<0).fillna(0)
test6c=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_iwv_merra2ac<=0.05).where(slopes_array_iwv_merra2ac<0).fillna(0)

test7c=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_iwv_jra55gw<=0.05).where(slopes_array_iwv_jra55gw<0).fillna(0)

test_allc=(test1c+test2c+test3c+test4c+test5c+test6c+test7c)/num_sets*100.
fieldc=test_allc

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)
test_allb=test_allb.where(num_sets>=2)
test_allc=test_allc.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))

cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1.,levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

X=4
for j in range(int(field.lat.size/X)):
    for k in range(int(field.lon.size/X)):
        if test_allb[X*j,X*k]>=66.66 or test_allc[X*j,X*k]>=66.66:
            ax.plot(field.lon[X*k],field.lat[X*j],marker='o',color='k',markersize=2,alpha=0.5)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{Trends:}$'+' '+r'$\bf{ARTMIP}$'+' '+r'$\bf{Tier}$'+' '+r'$\bf{2}$'+' '+r'$\bf{sample}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{IWV}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_sel_2.png')

In [ ]:
# datasets - 
# datasets - 3 AR variants
# datasets - 2 EIVT variants

test1=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_iwv_era5tp<=0.05).where(slopes_array_iwv_era5tp>0).fillna(0)
test2=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_iwv_era5reid<=0.05).where(slopes_array_iwv_era5reid>0).fillna(0)
test3=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_iwv_era5mk<=0.05).where(slopes_array_iwv_era5mk>0).fillna(0)

test4=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_iwv_merra2lo<=0.05).where(slopes_array_iwv_merra2lo>0).fillna(0)
test5=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_iwv_merra2cndl<=0.05).where(slopes_array_iwv_merra2cndl>0).fillna(0)
test6=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_iwv_merra2ac<=0.05).where(slopes_array_iwv_merra2ac>0).fillna(0)

test7=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_iwv_jra55gw<=0.05).where(slopes_array_iwv_jra55gw>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7)/num_sets*100.
field=test_all


test1a=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_iwv_era5tp<=0.05).where(slopes_array_iwv_era5tp<0).fillna(0)
test2a=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_iwv_era5reid<=0.05).where(slopes_array_iwv_era5reid<0).fillna(0)
test3a=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_iwv_era5mk<=0.05).where(slopes_array_iwv_era5mk<0).fillna(0)

test4a=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_iwv_merra2lo<=0.05).where(slopes_array_iwv_merra2lo<0).fillna(0)
test5a=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_iwv_merra2cndl<=0.05).where(slopes_array_iwv_merra2cndl<0).fillna(0)
test6a=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_iwv_merra2ac<=0.05).where(slopes_array_iwv_merra2ac<0).fillna(0)

test7a=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_iwv_jra55gw<=0.05).where(slopes_array_iwv_jra55gw<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)


field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))

cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1.,levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{Trends:}$'+' '+r'$\bf{ARTMIP}$'+' '+r'$\bf{Tier}$'+' '+r'$\bf{2}$'+' '+r'$\bf{sample}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{IWV}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_sel_2a.png')

In [ ]:
a=0.10#/sum_factor

p_values=sigs_array_s_era5tp.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_s_era5tp=sigs_array_s_era5tp.where(sigs_array_s_era5tp<=p_final)

p_values=sigs_array_s_era5reid.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_s_era5reid=sigs_array_s_era5reid.where(sigs_array_s_era5reid<=p_final)

p_values=sigs_array_s_era5mk.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_s_era5mk=sigs_array_s_era5mk.where(sigs_array_s_era5mk<=p_final)

########


p_values=sigs_array_s_merra2lo.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_s_merra2lo=sigs_array_s_merra2lo.where(sigs_array_s_merra2lo<=p_final)

p_values=sigs_array_s_merra2cndl.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_s_merra2cndl=sigs_array_s_merra2cndl.where(sigs_array_s_merra2cndl<=p_final)

p_values=sigs_array_s_merra2ac.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_s_merra2ac=sigs_array_s_merra2ac.where(sigs_array_s_merra2ac<=p_final)

######

p_values=sigs_array_s_jra55gw.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_s_jra55gw=sigs_array_s_jra55gw.where(sigs_array_s_jra55gw<=p_final)

In [ ]:
num_sets1=xr.ones_like(slopes_array_ivt_era5tp).where(slopes_array_s_era5tp>-1000)
num_sets1=num_sets1.fillna(0)
num_sets2=xr.ones_like(slopes_array_ivt_era5reid).where(slopes_array_s_era5reid>-1000)
num_sets2=num_sets2.fillna(0)
num_sets3=xr.ones_like(slopes_array_ivt_era5mk).where(slopes_array_s_era5mk>-1000)
num_sets3=num_sets3.fillna(0)

num_sets4=xr.ones_like(slopes_array_ivt_merra2lo).where(slopes_array_s_merra2lo>-1000)
num_sets4=num_sets4.fillna(0)
num_sets5=xr.ones_like(slopes_array_ivt_merra2cndl).where(slopes_array_s_merra2cndl>-1000)
num_sets5=num_sets5.fillna(0)
num_sets6=xr.ones_like(slopes_array_ivt_merra2ac).where(slopes_array_s_merra2ac>-1000)
num_sets6=num_sets6.fillna(0)

num_sets7=xr.ones_like(slopes_array_ivt_jra55gw).where(slopes_array_s_jra55gw>-1000)
num_sets7=num_sets7.fillna(0)

num_sets=num_sets1+num_sets2+num_sets3+num_sets4+num_sets5+num_sets6+num_sets7
field=num_sets


fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(0,10,1),cmap=plt.cm.Greens,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-2,2.2,0.2),cmap=plt.cm.seismic,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('# datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False
#ax.set_ylim(-60,70)

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{\#}$'+' '+r'$\bf{datasets:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{AR}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)

plt.show()

In [ ]:
# datasets - 
# datasets - 3 AR variants
# datasets - 2 EIVT variants

test1=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_s_era5tp<=0.05).where(slopes_array_s_era5tp>0).fillna(0)
test2=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_s_era5reid<=0.05).where(slopes_array_s_era5reid>0).fillna(0)
test3=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_s_era5mk<=0.05).where(slopes_array_s_era5mk>0).fillna(0)

test4=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_s_merra2lo<=0.05).where(slopes_array_s_merra2lo>0).fillna(0)
test5=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_s_merra2cndl<=0.05).where(slopes_array_s_merra2cndl>0).fillna(0)
test6=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_s_merra2ac<=0.05).where(slopes_array_s_merra2ac>0).fillna(0)

test7=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_final_s_jra55gw<=0.05).where(slopes_array_s_jra55gw>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7)/num_sets*100.
field=test_all


test1a=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_s_era5tp<=0.05).where(slopes_array_s_era5tp<0).fillna(0)
test2a=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_s_era5reid<=0.05).where(slopes_array_s_era5reid<0).fillna(0)
test3a=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_s_era5mk<=0.05).where(slopes_array_s_era5mk<0).fillna(0)

test4a=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_s_merra2lo<=0.05).where(slopes_array_s_merra2lo<0).fillna(0)
test5a=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_s_merra2cndl<=0.05).where(slopes_array_s_merra2cndl<0).fillna(0)
test6a=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_s_merra2ac<=0.05).where(slopes_array_s_merra2ac<0).fillna(0)

test7a=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_final_s_jra55gw<=0.05).where(slopes_array_s_jra55gw<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

#######

test1b=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_s_era5tp<=0.05).where(slopes_array_s_era5tp>0).fillna(0)
test2b=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_s_era5reid<=0.05).where(slopes_array_s_era5reid>0).fillna(0)
test3b=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_s_era5mk<=0.05).where(slopes_array_s_era5mk>0).fillna(0)

test4b=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_s_merra2lo<=0.05).where(slopes_array_s_merra2lo>0).fillna(0)
test5b=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_s_merra2cndl<=0.05).where(slopes_array_s_merra2cndl>0).fillna(0)
test6b=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_s_merra2ac<=0.05).where(slopes_array_s_merra2ac>0).fillna(0)

test7b=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_s_jra55gw<=0.05).where(slopes_array_s_jra55gw>0).fillna(0)

test_allb=(test1b+test2b+test3b+test4b+test5b+test6b+test7b)/num_sets*100.


test1c=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_s_era5tp<=0.05).where(slopes_array_s_era5tp<0).fillna(0)
test2c=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_s_era5reid<=0.05).where(slopes_array_s_era5reid<0).fillna(0)
test3c=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_s_era5mk<=0.05).where(slopes_array_s_era5mk<0).fillna(0)

test4c=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_s_merra2lo<=0.05).where(slopes_array_s_merra2lo<0).fillna(0)
test5c=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_s_merra2cndl<=0.05).where(slopes_array_s_merra2cndl<0).fillna(0)
test6c=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_s_merra2ac<=0.05).where(slopes_array_s_merra2ac<0).fillna(0)

test7c=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_s_jra55gw<=0.05).where(slopes_array_s_jra55gw<0).fillna(0)

test_allc=(test1c+test2c+test3c+test4c+test5c+test6c+test7c)/num_sets*100.
fieldc=test_allc

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)
test_allb=test_allb.where(num_sets>=2)
test_allc=test_allc.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))

cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1.,levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

X=4
for j in range(int(field.lat.size/X)):
    for k in range(int(field.lon.size/X)):
        if test_allb[X*j,X*k]>=66.66 or test_allc[X*j,X*k]>=66.66:
            ax.plot(field.lon[X*k],field.lat[X*j],marker='o',color='k',markersize=2,alpha=0.5)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{Trends:}$'+' '+r'$\bf{ARTMIP}$'+' '+r'$\bf{Tier}$'+' '+r'$\bf{2}$'+' '+r'$\bf{sample}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{S}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_sel_3.png')
#fig.savefig(dir_data+'titles_2.png')

In [ ]:
# datasets - 
# datasets - 3 AR variants
# datasets - 2 EIVT variants

test1=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_s_era5tp<=0.05).where(slopes_array_s_era5tp>0).fillna(0)
test2=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_s_era5reid<=0.05).where(slopes_array_s_era5reid>0).fillna(0)
test3=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_s_era5mk<=0.05).where(slopes_array_s_era5mk>0).fillna(0)

test4=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_s_merra2lo<=0.05).where(slopes_array_s_merra2lo>0).fillna(0)
test5=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_s_merra2cndl<=0.05).where(slopes_array_s_merra2cndl>0).fillna(0)
test6=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_s_merra2ac<=0.05).where(slopes_array_s_merra2ac>0).fillna(0)

test7=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_s_jra55gw<=0.05).where(slopes_array_s_jra55gw>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7)/num_sets*100.
field=test_all


test1a=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_s_era5tp<=0.05).where(slopes_array_s_era5tp<0).fillna(0)
test2a=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_s_era5reid<=0.05).where(slopes_array_s_era5reid<0).fillna(0)
test3a=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_s_era5mk<=0.05).where(slopes_array_s_era5mk<0).fillna(0)

test4a=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_s_merra2lo<=0.05).where(slopes_array_s_merra2lo<0).fillna(0)
test5a=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_s_merra2cndl<=0.05).where(slopes_array_s_merra2cndl<0).fillna(0)
test6a=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_s_merra2ac<=0.05).where(slopes_array_s_merra2ac<0).fillna(0)

test7a=xr.ones_like(sigs_array_ivt_jra55gw).where(sigs_array_s_jra55gw<=0.05).where(slopes_array_s_jra55gw<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)


field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))

cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1.,levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{Trends:}$'+' '+r'$\bf{ARTMIP}$'+' '+r'$\bf{Tier}$'+' '+r'$\bf{2}$'+' '+r'$\bf{sample}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{S}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_sel_3a.png')
#fig.savefig(dir_data+'titles_2.png')

In [ ]:
a=0.10#/sum_factor

p_values=sigs_array_mfc_era5tp.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_mfc_era5tp=sigs_array_mfc_era5tp.where(sigs_array_mfc_era5tp<=p_final)

p_values=sigs_array_mfc_era5reid.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_mfc_era5reid=sigs_array_mfc_era5reid.where(sigs_array_mfc_era5reid<=p_final)

p_values=sigs_array_mfc_era5mk.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_mfc_era5mk=sigs_array_mfc_era5mk.where(sigs_array_mfc_era5mk<=p_final)

########


p_values=sigs_array_mfc_merra2lo.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_mfc_merra2lo=sigs_array_mfc_merra2lo.where(sigs_array_mfc_merra2lo<=p_final)

p_values=sigs_array_mfc_merra2cndl.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_mfc_merra2cndl=sigs_array_mfc_merra2cndl.where(sigs_array_mfc_merra2cndl<=p_final)

p_values=sigs_array_mfc_merra2ac.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
    print(p_final)
except:
    p_final=np.nan
    print(p_final)
sigs_array_final_mfc_merra2ac=sigs_array_mfc_merra2ac.where(sigs_array_mfc_merra2ac<=p_final)

In [ ]:
num_sets1=xr.ones_like(slopes_array_ivt_era5tp).where(slopes_array_mfc_era5tp>-1000)
num_sets1=num_sets1.fillna(0)
num_sets2=xr.ones_like(slopes_array_ivt_era5reid).where(slopes_array_mfc_era5reid>-1000)
num_sets2=num_sets2.fillna(0)
num_sets3=xr.ones_like(slopes_array_ivt_era5mk).where(slopes_array_mfc_era5mk>-1000)
num_sets3=num_sets3.fillna(0)

num_sets4=xr.ones_like(slopes_array_ivt_merra2lo).where(slopes_array_mfc_merra2lo>-1000)
num_sets4=num_sets4.fillna(0)
num_sets5=xr.ones_like(slopes_array_ivt_merra2cndl).where(slopes_array_mfc_merra2cndl>-1000)
num_sets5=num_sets5.fillna(0)
num_sets6=xr.ones_like(slopes_array_ivt_merra2ac).where(slopes_array_mfc_merra2ac>-1000)
num_sets6=num_sets6.fillna(0)

num_sets=num_sets1+num_sets2+num_sets3+num_sets4+num_sets5+num_sets6
field=num_sets


fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(0,10,1),cmap=plt.cm.Greens,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-2,2.2,0.2),cmap=plt.cm.seismic,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('# datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False
#ax.set_ylim(-60,70)

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{\#}$'+' '+r'$\bf{datasets:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{AR}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)

plt.show()

In [ ]:
# datasets - 
# datasets - 3 AR variants
# datasets - 2 EIVT variants

test1=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_mfc_era5tp<=0.05).where(slopes_array_mfc_era5tp>0).fillna(0)
test2=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_mfc_era5reid<=0.05).where(slopes_array_mfc_era5reid>0).fillna(0)
test3=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_mfc_era5mk<=0.05).where(slopes_array_mfc_era5mk>0).fillna(0)

test4=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_mfc_merra2lo<=0.05).where(slopes_array_mfc_merra2lo>0).fillna(0)
test5=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_mfc_merra2cndl<=0.05).where(slopes_array_mfc_merra2cndl>0).fillna(0)
test6=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_mfc_merra2ac<=0.05).where(slopes_array_mfc_merra2ac>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6)/num_sets*100.
field=test_all


test1a=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_final_mfc_era5tp<=0.05).where(slopes_array_mfc_era5tp<0).fillna(0)
test2a=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_final_mfc_era5reid<=0.05).where(slopes_array_mfc_era5reid<0).fillna(0)
test3a=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_final_mfc_era5mk<=0.05).where(slopes_array_mfc_era5mk<0).fillna(0)

test4a=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_final_mfc_merra2lo<=0.05).where(slopes_array_mfc_merra2lo<0).fillna(0)
test5a=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_final_mfc_merra2cndl<=0.05).where(slopes_array_mfc_merra2cndl<0).fillna(0)
test6a=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_final_mfc_merra2ac<=0.05).where(slopes_array_mfc_merra2ac<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

#######

test1b=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_mfc_era5tp<=0.05).where(slopes_array_mfc_era5tp>0).fillna(0)
test2b=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_mfc_era5reid<=0.05).where(slopes_array_mfc_era5reid>0).fillna(0)
test3b=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_mfc_era5mk<=0.05).where(slopes_array_mfc_era5mk>0).fillna(0)

test4b=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_mfc_merra2lo<=0.05).where(slopes_array_mfc_merra2lo>0).fillna(0)
test5b=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_mfc_merra2cndl<=0.05).where(slopes_array_mfc_merra2cndl>0).fillna(0)
test6b=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_mfc_merra2ac<=0.05).where(slopes_array_mfc_merra2ac>0).fillna(0)

test_allb=(test1b+test2b+test3b+test4b+test5b+test6b)/num_sets*100.


test1c=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_mfc_era5tp<=0.05).where(slopes_array_mfc_era5tp<0).fillna(0)
test2c=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_mfc_era5reid<=0.05).where(slopes_array_mfc_era5reid<0).fillna(0)
test3c=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_mfc_era5mk<=0.05).where(slopes_array_mfc_era5mk<0).fillna(0)

test4c=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_mfc_merra2lo<=0.05).where(slopes_array_mfc_merra2lo<0).fillna(0)
test5c=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_mfc_merra2cndl<=0.05).where(slopes_array_mfc_merra2cndl<0).fillna(0)
test6c=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_mfc_merra2ac<=0.05).where(slopes_array_mfc_merra2ac<0).fillna(0)

test_allc=(test1c+test2c+test3c+test4c+test5c+test6c)/num_sets*100.
fieldc=test_allc

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)
test_allb=test_allb.where(num_sets>=2)
test_allc=test_allc.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))

cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1.,levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

X=4
for j in range(int(field.lat.size/X)):
    for k in range(int(field.lon.size/X)):
        if test_allb[X*j,X*k]>=66.66 or test_allc[X*j,X*k]>=66.66:
            ax.plot(field.lon[X*k],field.lat[X*j],marker='o',color='k',markersize=2,alpha=0.5)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{Trends:}$'+' '+r'$\bf{ARTMIP}$'+' '+r'$\bf{Tier}$'+' '+r'$\bf{2}$'+' '+r'$\bf{sample}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{MFC}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_sel_4.png')

In [ ]:
# datasets - 
# datasets - 3 AR variants
# datasets - 2 EIVT variants

test1=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_mfc_era5tp<=0.05).where(slopes_array_mfc_era5tp>0).fillna(0)
test2=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_mfc_era5reid<=0.05).where(slopes_array_mfc_era5reid>0).fillna(0)
test3=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_mfc_era5mk<=0.05).where(slopes_array_mfc_era5mk>0).fillna(0)

test4=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_mfc_merra2lo<=0.05).where(slopes_array_mfc_merra2lo>0).fillna(0)
test5=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_mfc_merra2cndl<=0.05).where(slopes_array_mfc_merra2cndl>0).fillna(0)
test6=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_mfc_merra2ac<=0.05).where(slopes_array_mfc_merra2ac>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6)/num_sets*100.
field=test_all


test1a=xr.ones_like(sigs_array_ivt_era5tp).where(sigs_array_mfc_era5tp<=0.05).where(slopes_array_mfc_era5tp<0).fillna(0)
test2a=xr.ones_like(sigs_array_ivt_era5reid).where(sigs_array_mfc_era5reid<=0.05).where(slopes_array_mfc_era5reid<0).fillna(0)
test3a=xr.ones_like(sigs_array_ivt_era5mk).where(sigs_array_mfc_era5mk<=0.05).where(slopes_array_mfc_era5mk<0).fillna(0)

test4a=xr.ones_like(sigs_array_ivt_merra2lo).where(sigs_array_mfc_merra2lo<=0.05).where(slopes_array_mfc_merra2lo<0).fillna(0)
test5a=xr.ones_like(sigs_array_ivt_merra2cndl).where(sigs_array_mfc_merra2cndl<=0.05).where(slopes_array_mfc_merra2cndl<0).fillna(0)
test6a=xr.ones_like(sigs_array_ivt_merra2ac).where(sigs_array_mfc_merra2ac<=0.05).where(slopes_array_mfc_merra2ac<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)



field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))

cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1.,levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{Trends:}$'+' '+r'$\bf{ARTMIP}$'+' '+r'$\bf{Tier}$'+' '+r'$\bf{2}$'+' '+r'$\bf{sample}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{MFC}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_sel_4a.png')